In [1]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import FeatureAgglomeration
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
import math
import pickle
import os

## Import custom functions from 'pre_processing.py' ##
try:
    import importlib
    importlib.reload(pp)
except NameError: # It hasn't been imported yet
    import pre_processing as pp
    
fnirs_data = r'C:\Users\61407\mental-health-modelling\data\fNIRS\bandpass_filtered_whole_data'

## 0. Data restructuring ##
- Split the data into task blocks (16 blocks per subject, each block consists of data from one level of task difficulty), as they are independent time series (and we don't want overlap between task blocks)
- Further split the 80 second task blocks into "chunks" of 10 seconds each, to increase number of samples (8 chunks x 16 blocks = 128 samples per subject). These chunks of data are what will eventually be fed into the models

In [2]:
structure_suffix = '20s_chunks_no_overlap' # suffix to add at end of model name, to indicate how data was restructure

In [3]:
chunk_seconds = 20 #how long you want each chunk to be in seconds
overlap_seconds = 0 #how many seconds of overlap you want across chunks

## Upload raw data from repository ##
all_data = pp.upload_all_subjects(fnirs_data, scaled=False)

# First split data into task blocks by assigning a block label
all_grouped_dataframes, all_block_change_idx = pp.detect_block_changes(all_data, label='difficulty')
block_labelled_data = pd.concat(all_grouped_dataframes)

# Make sure that every block is the same size, otherwise the following line will raise an error
block_size = int(block_labelled_data.groupby(block_labelled_data.index.get_level_values(0)).size().unique())

# Split data from task blocks into 10 second chunks by assigning a chunk label
chunk_labelled_data = block_labelled_data.groupby(block_labelled_data.index.get_level_values(0)).apply(pp.split_chunks, chunk_seconds)

# Do some index restructuring for aesthetics - this is a messy code that should be optimized
chunk_labelled_data = chunk_labelled_data.droplevel(0).reset_index().set_index(['level_0','level_1','level_2','level_3'])
chunk_labelled_data.index.names = ['chunk','block','subject','sample']
unique_chunk = np.array([[i+1]*math.floor(chunk_seconds*5.2) for i, (x) in enumerate(chunk_labelled_data.droplevel([2,3]).index.unique())]).flatten()
chunk_labelled_data['unique_chunk_ID'] = unique_chunk
chunk_labelled_data = chunk_labelled_data.reset_index().set_index(['unique_chunk_ID','block','chunk','subject','sample'])

## 1. Define paragidms and data partitioning ##
***Generic model:*** The first paradigm is to try and build a "one size fits all" model that is trained on many subjects, and can generalize to new subject data. This would allow for a pre-trained model to be deployed with the software, and there will be **no model training required at the user end**. However, this would the software to cognitive workload-related tasks only, and have **limited flexibility in its applications**. Application flexibility could be increased by deploying multiple pre-trained models for different tasks, but this would require extensive and continuous data collection.
- *Data partitioning for model training will involve randomly splitting the 68 subjects into 51 training (75%) and 17 testing (25%) subjects. We will perform this split in 4 different "buckets", where every bucket will have different test subjects from each other (random selection of 17 test subjects without replacement for each bucket).*

***Subject-specific models:*** The second paradigm is to build a single model for each subject, where only a model architecture will be deployed with the software, and **model training will be required at the user end for optimiation**. This approach would **increase the flexibility of its applications beyond cognitive workload-related tasks**, where a user could train the model architecture on specific tasks that are relevant to their daily lives. This approach would probably **require a cloud interface on the developer end to accept model training requests from users**, as model training may require high computational capacity that users may not have on their local device.
- *Data partitioning for model training will involve splitting each subject's data into 12 training and 4 test blocks, where there will be one block from each level of task difficulty (4 difficulties) in the test blocks. This results in 96 training and 32 testing "chunks".*

**1.1 Partition the raw data for generic model building**
- Here we will only extract the indexes of partitions for each bucket, to avoid having to write the data partitions into memory (and will use these indexes and reference the entire data set at later steps)

In [4]:
generic_suffix = '51train_17test' # suffix to add at end of model name, to indicate how generic models were partitioned

In [5]:
# Define the desired number of test subjects in each bucket
test_size = 17

# Total number of subjects
n_subjects = all_data.index.get_level_values(0).unique().shape[0]

# Run random partitioning to get test bucket index positions
seed = 42 # for reproducibility
test_buckets_positions = pp.partition_subject_indexes(test_size, n_subjects, seed)

# Get the actual index names of test buckets based on dataframe
test_buckets = [all_data.index.get_level_values(0).unique()[x] for x in test_buckets_positions]

# Get the train buckets
train_buckets = [all_data[~all_data.index.get_level_values(0).isin(x)].index.get_level_values(0).unique() for x in test_buckets]

4 buckets
68 unique indexes
Repeated indexes: (array([], dtype=int64),)


**1.2 Partition the raw data for subject-specific model building**
- Here we will only define the test indexes, and use a boolean operation to partition into train/test in later steps

In [6]:
subject_suffix = '1test_per_difficulty' # suffix to add at end of model name, to indicate how subject models were partitioned

In [7]:
# Create groupby to get the block numbers for each task difficulty, for each subject
groups = chunk_labelled_data.groupby([chunk_labelled_data.index.get_level_values('subject'),
                                     'difficulty',
                                     ]
                                    )

# Randomly sample one block from each groupby group {subject/difficulty : block number}
test_idx = {name:np.random.choice(indices)[1] for name, indices in groups.groups.items()}

## 2. Feature engineering (to be applied for both paradigms) ##

**2.1 Summary sensor metrics across time series samples (i.e. remove temporal information)**
 - Use the mean, median, std, min, max, RMS, skewness of each time series
 - Apply this feature engineering on a **chunk-by-chunk basis**, as they are the "independent" samples to feed into the models

In [8]:
# Define a dictionary of aggregation functions for each metric
def rms(series): # root mean square
    return np.sqrt(np.mean(series**2))

agg_functions = {'mean':'mean',
                 'median':'median',
                 'std':'std',
                 'min':'min',
                 'max':'max',
                 'rms':rms,
                 'skew':'skew'}

# Define a function to retrieve unique column values (for label extraction)
def get_unique_column_values(group, column, dtype):
    return dtype(group[column].unique())

# First calculate aggregate functions on each block of time series
agg = {key: chunk_labelled_data.drop('difficulty', axis=1).groupby(chunk_labelled_data.drop('difficulty', axis=1).index.get_level_values(0)).agg(value)
              for key, value in agg_functions.items()}
chunked_data = pd.concat(agg, axis=1)

# Get indexes for each chunk
chunked_indexes = [chunk_labelled_data[chunk_labelled_data.index.get_level_values('unique_chunk_ID') == x].index[0][:-1] for x in chunked_data.index]
chunked_multiindex = pd.MultiIndex.from_tuples(chunked_indexes)
chunked_data.index = chunked_multiindex
chunked_data.index.names = ['unique_chunk_ID','block','chunk','subject']

**2.1 Label engineering**
- Decide whether we want to predict all labels, a subset of labels (e.g. level 0 vs. level 2), or combined labels (e.g. "low" difficulty levels 0 + 1 vs. "high" difficulty levels 2 + 3).
- Each type of label engineering will have different use cases

In [9]:
label_suffix = 'low_vs_high' # 0 = low (level 0 or 1), 1 = high (level 2 or 3)

# Get labels for each chunk
chunked_labels = chunk_labelled_data.groupby(chunk_labelled_data.index.get_level_values(0)).apply(get_unique_column_values, 'difficulty', int)
chunked_labels.index = chunked_data.index

# Apply label engineering
chunked_labels = chunked_labels.replace({1:0,2:1,3:1})
# chunked_labels = chunked_labels[chunked_labels.isin([0,2])]
# chunked_data = chunked_data[chunked_data.index.isin(chunked_labels.index)]

## 3. Define options for pipeline components and hyperparameter search spaces ##

**3.1. Define pipeline components**

In [10]:
## Normalization, dimensionality reduction, and classifier options to test ##
norm = [StandardScaler()]
dim = [PCA(n_components=0.9), FeatureAgglomeration(n_clusters=4)]
clf = [SVC(), RandomForestClassifier(), KNeighborsClassifier()]

**3.2. Define classifier hyperparameter search spaces**

In [11]:
# Support vector classifier search space
svc_params = {
    'clf__C': [0.1, 1, 10, 100],  # Regularization parameter
    'clf__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Kernel type
    'clf__gamma': ['scale', 'auto', 0.1, 1],  # Kernel coefficient for 'rbf', 'poly' and 'sigmoid'
    'clf__degree': [2, 3, 4],  # Degree of the polynomial kernel ('poly')
    'clf__coef0': [0.0, 1.0, 2.0],  # Independent term in the kernel function
}

# Random forest classifier search space
rf_params = {
    'clf__n_estimators': [50, 100, 200],  # Number of trees in the forest
    'clf__max_features': ['auto', 'sqrt', 'log2', None],  # Number of features to consider at each split
    'clf__max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'clf__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'clf__min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
    'clf__bootstrap': [True, False],  # Method of selecting samples for training each tree
}

# K-nearest neighbors classifier search space
knn_params = {
    'clf__n_neighbors': [3, 5, 10, 20],  # Number of neighbors
    'clf__weights': ['uniform', 'distance'],  # Weight function used in prediction
    'clf__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithm used to compute the nearest neighbors
    'clf__p': [1, 2],  # Power parameter for the Minkowski metric
}

**3.3. Combine together all possible combinations of pipelines**

In [12]:
pipelines = {}
p = 1
for n in norm:
    for d in dim:
        for c in clf:
            if d is not None:
                steps = [('normalization',n), ('dim_reduce',d), ('clf',c)]
            else:
                steps = [('normalization',n), ('clf',c)]                
            pipelines[f'Pipeline_{p}'] = Pipeline(steps)
            p += 1

## 4. Partition, create pipeline, and CV search hyperparemeters for generic paradigm ##
- Here we are using the training data only to optimize the best hyperparameters for each pipeline
- The best estimator returned from the randomized search CV object is a pipeline that contains all pre-processing steps, so all the "fit_transform" methods will be appropriately applied to test data, when calling the "score" method.

In [13]:
# Dirctory of where you want to write/read these models
generic_ML_dir = r'C:\Users\61407\mental-health-modelling\fNIRS Mental Workload\final_models\generic'

for bucket, (test, train) in enumerate(zip(test_buckets, train_buckets)):
    print(f'Bucket {bucket}:\n{"*"*120}')
    
    # Name of the model to save
    fname = f'generic_ML_bucket{bucket}_{structure_suffix}_{generic_suffix}_{label_suffix}.pkl'
    
    # Check if model already exists
    if fname in os.listdir(generic_ML_dir):
        print(f'Already exists!\n{"*"*120}\n')
    else:
    
        # Data partitioning
        print('partitioning...')
        X_train, X_test = chunked_data[chunked_data.index.get_level_values('subject').isin(train)], chunked_data[chunked_data.index.get_level_values('subject').isin(test)]
        y_train, y_test = chunked_labels[chunked_labels.index.get_level_values('subject').isin(train)], chunked_labels[chunked_labels.index.get_level_values('subject').isin(test)]
        print('...complete!\n')

        # Iterate through different pipelines
        pipeline_results = {}
        for name, pipe in pipelines.items():
            pipe_clf = pipe[-1] 

            if isinstance(pipe_clf, KNeighborsClassifier):
                param_dist = knn_params
            elif isinstance(pipe_clf, RandomForestClassifier):
                param_dist = rf_params
            elif isinstance(pipe_clf, SVC):
                param_dist = svc_params
            else:
                print('classifier instance unknown')
                break

            # Set up randomized search CV
            random_search = RandomizedSearchCV(
            pipe,  # The pipeline containing the classifier and other steps
            param_distributions=param_dist,  # The hyperparameter search space
            n_iter=15,  # Number of parameter settings to sample
            cv=3,  # Number of cross-validation folds
            scoring='accuracy',  # Scoring metric
            random_state=42,  # Random state for reproducibility
            verbose=2,
            n_jobs=-1  # Number of CPU cores to use (set to -2 to use all available but one)
            )

            # Fit the RandomizedSearchCV object to the data
            print(f'{name}...\n{pipe.steps}\n')
            random_search.fit(X_train.to_numpy(), y_train.to_numpy())

            # Get the best hyperparameters and the corresponding estimator
            best_params = random_search.best_params_
            best_estimator = random_search.best_estimator_

            # Print the best hyperparameters for the current pipeline
            print("Best hyperparameters:", best_params)

            # Print the average validation accuracy for best hyperparameters
            avg_val = round(random_search.best_score_,3)
            print(f'Average validation accuracy: {avg_val}')

            # Get predictions on test data
            predictions = {}
            for sub in y_test.index.get_level_values('subject').unique():
                X_sub = X_test[X_test.index.get_level_values('subject') == sub].to_numpy()
                y_sub = y_test[y_test.index.get_level_values('subject') ==  sub].to_numpy()
                sub_predictions = best_estimator.predict(X_sub)
                predictions[sub] = {'ytrue':y_sub, 'ypred':sub_predictions}
            
            # Score test predictions
            test_score = round(best_estimator.score(X_test.to_numpy(), y_test.to_numpy()),3)
            print(f'Test accuracy: {test_score}')

            # Add random search object for current pipeline to dictionary
            pipeline_results[name] = {}
            pipeline_results[name]['predictions'] = predictions
            pipeline_results[name]['test_score'] = test_score
            pipeline_results[name]['rcv'] = random_search
        
        # Save bucket pipelines
        with open(os.path.join(generic_ML_dir, fname), 'wb') as file:
            pickle.dump(pipeline_results, file)
        print(f'{"*"*120}\n')

Bucket 0:
************************************************************************************************************************
partitioning...
...complete!

Pipeline_1...
[('normalization', StandardScaler()), ('dim_reduce', PCA(n_components=0.9)), ('clf', SVC())]

Fitting 3 folds for each of 15 candidates, totalling 45 fits


KeyboardInterrupt: 

## 5. Same as 4. but for subject-by-subject paradigm ##

In [109]:
# Dirctory of where you want to write/read these models
subject_ML_dir = r'C:\Users\61407\mental-health-modelling\fNIRS Mental Workload\final_models\subject'

for sub in chunked_data.index.get_level_values('subject').unique():
    print(f'{sub}:\n{"*"*120}')
    fname = f'{sub}_ML_{structure_suffix}_{subject_suffix}_{label_suffix}.pkl'
    
    # Check if model already exists
    if fname in os.listdir(subject_ML_dir):
        print(f'Already exists!\n{"*"*120}\n')
    else:
        
        # Data partitioning
        print('partitioning...')
        X_test = chunked_data[(chunked_data.index.get_level_values('subject') == sub) &
                              (chunked_data.index.get_level_values('block').isin([value for key, value in test_idx.items() if key[0] == sub]))
                             ]
        y_test = chunked_labels[(chunked_labels.index.get_level_values('subject') == sub) &
                                (chunked_labels.index.get_level_values('block').isin([value for key, value in test_idx.items() if key[0] == sub]))
                               ]
        X_train = chunked_data[(chunked_data.index.get_level_values('subject') == sub) &
                                        (~chunked_data.index.isin(X_test.index))]
        y_train = chunked_labels[(chunked_labels.index.get_level_values('subject') == sub) &
                                 (~chunked_labels.index.isin(y_test.index))]
        print('...complete!\n')

        # Iterate through different pipelines
        pipeline_results = {}
        for name, pipe in pipelines.items():
            pipe_clf = pipe[-1] 

            if isinstance(pipe_clf, KNeighborsClassifier):
                param_dist = knn_params
            elif isinstance(pipe_clf, RandomForestClassifier):
                param_dist = rf_params
            elif isinstance(pipe_clf, SVC):
                param_dist = svc_params
            else:
                print('classifier instance unknown')
                break

            # Set up randomized search CV
            random_search = RandomizedSearchCV(
            pipe,  # The pipeline containing the classifier and other steps
            param_distributions=param_dist,  # The hyperparameter search space
            n_iter=15,  # Number of parameter settings to sample
            cv=3,  # Number of cross-validation folds
            scoring='accuracy',  # Scoring metric
            random_state=42,  # Random state for reproducibility
            verbose=2,
            n_jobs=-1  # Number of CPU cores to use (set to -2 to use all available but one)
            )

            # Fit the RandomizedSearchCV object to the data
            print(f'{name}...\n{pipe.steps}\n')
            random_search.fit(X_train, y_train)

            # Get the best hyperparameters and the corresponding estimator
            best_params = random_search.best_params_
            best_estimator = random_search.best_estimator_

            # Print the best hyperparameters for the current pipeline
            print("Best hyperparameters:", best_params)

            # Print the average validation accuracy for best hyperparameters
            avg_val = round(random_search.best_score_,3)
            print(f'Average validation accuracy: {avg_val}')

            # Get predictions on test data using the best hyperparameter settings
            y_test_pred = best_estimator.predict(X_test)
            test_score = best_estimator.score(X_test, y_test)
            print(f'Test accuracy: {test_score}')      

            # Add random search object for current pipeline to dictionary
            pipeline_results[name] = {}
            pipeline_results[name]['best_test'] = {'y_true':y_test, 'y_pred':y_test_pred, 'accuracy':test_score}
            pipeline_results[name]['rcv'] = random_search
        
        # Save subject pipelines
        with open(os.path.join(subject_ML_dir, fname), 'wb') as file:
            pickle.dump(pipeline_results, file)
        print(f'{"*"*120}\n')

sub_1:
************************************************************************************************************************
partitioning...
...complete!

Pipeline_1...
[('normalization', StandardScaler()), ('dim_reduce', PCA(n_components=0.9)), ('clf', SVC())]

Fitting 3 folds for each of 15 candidates, totalling 45 fits
Best hyperparameters: {'clf__kernel': 'poly', 'clf__gamma': 'auto', 'clf__degree': 3, 'clf__coef0': 2.0, 'clf__C': 0.1}
Average validation accuracy: 0.75
Test accuracy: 1.0
Pipeline_2...
[('normalization', StandardScaler()), ('dim_reduce', PCA(n_components=0.9)), ('clf', RandomForestClassifier())]

Fitting 3 folds for each of 15 candidates, totalling 45 fits
Best hyperparameters: {'clf__n_estimators': 100, 'clf__min_samples_split': 10, 'clf__min_samples_leaf': 1, 'clf__max_features': 'sqrt', 'clf__max_depth': 20, 'clf__bootstrap': False}
Average validation accuracy: 0.75
Test accuracy: 0.875
Pipeline_3...
[('normalization', StandardScaler()), ('dim_reduce', PCA(n_c

C:\Users\61407\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [0.70833333 0.75       0.66666667        nan 0.70833333 0.70833333
 0.66666667        nan 0.75              nan 0.75       0.75
 0.75              nan 0.625     ]
  warnings.warn(


Best hyperparameters: {'clf__kernel': 'poly', 'clf__gamma': 'scale', 'clf__degree': 3, 'clf__coef0': 2.0, 'clf__C': 10}
Average validation accuracy: 0.917
Test accuracy: 1.0
Pipeline_5...
[('normalization', StandardScaler()), ('dim_reduce', FeatureAgglomeration(n_clusters=4)), ('clf', RandomForestClassifier())]

Fitting 3 folds for each of 15 candidates, totalling 45 fits
Best hyperparameters: {'clf__n_estimators': 50, 'clf__min_samples_split': 10, 'clf__min_samples_leaf': 2, 'clf__max_features': 'log2', 'clf__max_depth': 30, 'clf__bootstrap': True}
Average validation accuracy: 0.792
Test accuracy: 1.0
Pipeline_6...
[('normalization', StandardScaler()), ('dim_reduce', FeatureAgglomeration(n_clusters=4)), ('clf', KNeighborsClassifier())]

Fitting 3 folds for each of 15 candidates, totalling 45 fits
Best hyperparameters: {'clf__weights': 'uniform', 'clf__p': 1, 'clf__n_neighbors': 3, 'clf__algorithm': 'auto'}
Average validation accuracy: 0.875
Test accuracy: 1.0
*************************

C:\Users\61407\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [0.79166667 0.75       0.875             nan 0.79166667 0.79166667
 0.875             nan 0.75              nan 0.75       0.75
 0.75              nan 0.79166667]
  warnings.warn(


Best hyperparameters: {'clf__kernel': 'poly', 'clf__gamma': 'auto', 'clf__degree': 3, 'clf__coef0': 2.0, 'clf__C': 0.1}
Average validation accuracy: 0.667
Test accuracy: 0.5
Pipeline_2...
[('normalization', StandardScaler()), ('dim_reduce', PCA(n_components=0.9)), ('clf', RandomForestClassifier())]

Fitting 3 folds for each of 15 candidates, totalling 45 fits
Best hyperparameters: {'clf__n_estimators': 200, 'clf__min_samples_split': 5, 'clf__min_samples_leaf': 2, 'clf__max_features': None, 'clf__max_depth': 10, 'clf__bootstrap': False}
Average validation accuracy: 0.583
Test accuracy: 0.375
Pipeline_3...
[('normalization', StandardScaler()), ('dim_reduce', PCA(n_components=0.9)), ('clf', KNeighborsClassifier())]

Fitting 3 folds for each of 15 candidates, totalling 45 fits
Best hyperparameters: {'clf__weights': 'uniform', 'clf__p': 1, 'clf__n_neighbors': 5, 'clf__algorithm': 'brute'}
Average validation accuracy: 0.625
Test accuracy: 0.5
Pipeline_4...
[('normalization', StandardScaler()

...

C:\Users\61407\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [0.375      0.45833333 0.54166667        nan 0.375      0.375
 0.54166667        nan 0.54166667        nan 0.625      0.54166667
 0.625             nan 0.54166667]
  warnings.warn(


Best hyperparameters: {'clf__weights': 'uniform', 'clf__p': 1, 'clf__n_neighbors': 10, 'clf__algorithm': 'brute'}
Average validation accuracy: 0.625
Test accuracy: 0.75
Pipeline_4...
[('normalization', StandardScaler()), ('dim_reduce', FeatureAgglomeration(n_clusters=4)), ('clf', SVC())]

Fitting 3 folds for each of 15 candidates, totalling 45 fits
Best hyperparameters: {'clf__kernel': 'sigmoid', 'clf__gamma': 1, 'clf__degree': 2, 'clf__coef0': 0.0, 'clf__C': 100}
Average validation accuracy: 0.625
Test accuracy: 0.625
Pipeline_5...
[('normalization', StandardScaler()), ('dim_reduce', FeatureAgglomeration(n_clusters=4)), ('clf', RandomForestClassifier())]

Fitting 3 folds for each of 15 candidates, totalling 45 fits


C:\Users\61407\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best hyperparameters: {'clf__n_estimators': 200, 'clf__min_samples_split': 5, 'clf__min_samples_leaf': 4, 'clf__max_features': 'auto', 'clf__max_depth': None, 'clf__bootstrap': True}
Average validation accuracy: 0.458
Test accuracy: 0.625
Pipeline_6...
[('normalization', StandardScaler()), ('dim_reduce', FeatureAgglomeration(n_clusters=4)), ('clf', KNeighborsClassifier())]

Fitting 3 folds for each of 15 candidates, totalling 45 fits
Best hyperparameters: {'clf__weights': 'uniform', 'clf__p': 2, 'clf__n_neighbors': 10, 'clf__algorithm': 'brute'}
Average validation accuracy: 0.625
Test accuracy: 0.875
************************************************************************************************************************



C:\Users\61407\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [0.54166667 0.625      0.58333333        nan 0.54166667 0.54166667
 0.58333333        nan 0.54166667        nan 0.5        0.54166667
 0.5               nan 0.58333333]
  warnings.warn(
